In [1]:
# 128 bit AES algorithm
# created on 29th April 2022
# author - kithekadankioko@gmail.com



In [623]:
import numpy as np
import pandas as pd

class AES_CIPHER:
    ''' this class implements the 128 bit AES cipher'''
    
    def __init__(self, plaintext, key):
        ''' initialize a text and key to encrypt'''
        
        self._plaintext = plaintext
        self._key = key
        
    def divideIntoBlocks(self, text):
        '''
        takes in text -> a string o
    
        text is divided into blocks of length of 4 words/16 bytes/128 bits
    
        output is a vector of numpy block arrays, with the last block padded with spaces to fit the shape
        '''
        ascii_text = [ord(letter) for letter in text]
        length = len(text)
        mod = length%16
        if length%16 == 0:   
            # checks whether text already perfect length
            output = [np.array(ascii_text[i:16+i]).reshape(4,4) for i in range(0,length,16)]
        elif length > 16:# asign space as padding to the trailing remainder 
            blocks = [np.array(ascii_text[i:16+i]).reshape(4,4) for i in range(0,length-mod,16)]
            temp = np.array(ascii_text[-mod:])
            blocks.append((np.pad(temp,pad_width=(0,16-mod),constant_values=(0)).reshape(4,4)))
            output  = [np.transpose(block) for block in blocks]
        else:
            output = np.pad(ascii_text, pad_width=(0,16-length),constant_values=(0).reshape(4,4))
        return output
    
    def multiply(self, x, y):
        ''' takes in 2 values of 8-bits
        does multiplication in the gf(2^3) field'''
        p = 0b100011011      # modulo x^8+x^4+x^3+x+1
        m = 0                # m is where we store the product
        for i in range(8):
            m = m<<1
            if m & 0b100000000:
                m = m^p
            if y & 0b010000000:
                m= m^x
            y = y << 1
        return m
    
    def mixColumns(self, block):
        '''
        block is a 4,4 matrix 
        the output is the block multiplied by a constant matrix
        '''
        ouput = [[],[],[],[]]
        ouput[0][:] = [self.multiply(0b10,block[i][0])^
                       self.multiply(0b11,block[i][1])^
                       block[i][2]^
                       block[i][3] 
                       for i in range(4)]
        ouput[1][:] = [block[i][0]^
                       self.multiply(0b10,block[i][1])^
                       self.multiply(0b11,block[i][2])^
                       block[i][3] 
                       for i in range(4)]
        ouput[2][:] = [block[i][0]^
                       block[i][1]^
                       self.multiply(0b10,block[i][2])^
                       self.multiply(0b11,block[i][3]) 
                       for i in range(4)]
        ouput[3][:] = [self.multiply(0b11,block[i][0])^
                       block[i][1]^
                       block[i][2]^
                       self.multiply(0b10,block[i][3]) 
                       for i in range(4)]
        return np.transpose(ouput)
    
    
    def invMixColumns(self, block):
        '''
         block is a 4,4 matrix
         ouput is the block multiplied by a constant matrix
    '''
        ouput = [[],[],[],[]]
        ouput[0][:] = [self.multiply(0x0e,block[i][0])^
                       self.multiply(0x0b,block[i][1])^
                       self.multiply(0x0d,block[i][2])^
                       self.multiply(0x09,block[i][3]) 
                       for i in range(4)]
        ouput[1][:] = [self.multiply(0x09,block[i][0])^
                       self.multiply(0x0e,block[i][1])^
                       self.multiply(0x0b,block[i][2])^
                       self.multiply(0x0d,block[i][3])
                       for i in range(4)]
        ouput[2][:] = [self.multiply(0x0d,block[i][0])^
                       self.multiply(0x09,block[i][1])^
                       self.multiply(0x0e,block[i][2])^
                       self.multiply(0x0b,block[i][3])
                       for i in range(4)]
        ouput[3][:] = [self.multiply(0x0b,block[i][0])^
                       self.multiply(0x0d,block[i][1])^
                       self.multiply(0x09,block[i][2])^
                       self.multiply(0x0e,block[i][3]) 
                       for i in range(4)]
            
        return np.transpose(ouput)
        
   
    def subWord(self, state):
        '''
    input is a (4,4)shape vector.
    each element in input is a byte in decimal form
    output is a (4,4) shape transformed vector. 4 words(columns)
    '''
        data = [
    [0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76],
    [0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0],
    [0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15],
    [0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75],
    [0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84],
    [0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf],
    [0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8],
    [0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2],
    [0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73],
    [0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb],
    [0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79],
    [0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08],
    [0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a],
    [0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e],
    [0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf],
    [0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16],
          ]
    
        SBOX = pd.DataFrame(data, index=['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f'],
                   columns =['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f'])
        if state.shape == (4,4):
            transformed_state = [[SBOX.loc[hex(word)[-2],hex(word)[-1]]  
                         if len(hex(word))==4
                         else SBOX.loc['0',hex(word)[-1]]
                         for word in state[i] ] for i in range(4)]
        elif state.shape == (4,):
            transformed_state = [SBOX.loc[hex(word)[-2],hex(word)[-1]]  
                         if len(hex(word))==4
                         else SBOX.loc['0',hex(word)[-1]]
                         for word in state ]

        return np.array(transformed_state)
    
    def invSubWord(self, state):
        '''
        input is a (4,)shape vector.
        each element in input is a byte in decimal form
        output is a (4,) shape transformed vector. 4 words(columns)
        '''
        data = [
    [0x52,0x09,0x6a,0xd5,0x30,0x36,0xa5,0x38,0xbf,0x40,0xa3,0x9e,0x81,0xf3,0xd7,0xfb],
    [0x7c,0xe3,0x39,0x82,0x9b,0x2f,0xff,0x87,0x34,0x8e,0x43,0x44,0xc4,0xde,0xe9,0xcb],
    [0x54,0x7b,0x94,0x32,0xa6,0xc2,0x23,0x3d,0xee,0x4c,0x95,0x0b,0x42,0xfa,0xc3,0x4e],
    [0x08,0x2e,0xa1,0x66,0x28,0xd9,0x24,0xb2,0x76,0x5b,0xa2,0x49,0x6d,0x8b,0xd1,0x25],
    [0x72,0xf8,0xf6,0x64,0x86,0x68,0x98,0x16,0xd4,0xa4,0x5c,0xcc,0x5d,0x65,0xb6,0x92],
    [0x6c,0x70,0x48,0x50,0xfd,0xed,0xb9,0xda,0x5e,0x15,0x46,0x57,0xa7,0x8d,0x9d,0x84],
    [0x90,0xd8,0xab,0x00,0x8c,0xbc,0xd3,0x0a,0xf7,0xe4,0x58,0x05,0xb8,0xb3,0x45,0x06],
    [0xd0,0x2c,0x1e,0x8f,0xca,0x3f,0x0f,0x02,0xc1,0xaf,0xbd,0x03,0x01,0x13,0x8a,0x6b],
    [0x3a,0x91,0x11,0x41,0x4f,0x67,0xdc,0xea,0x97,0xf2,0xcf,0xce,0xf0,0xb4,0xe6,0x73],
    [0x96,0xac,0x74,0x22,0xe7,0xad,0x35,0x85,0xe2,0xf9,0x37,0xe8,0x1c,0x75,0xdf,0x6e],
    [0x47,0xf1,0x1a,0x71,0x1d,0x29,0xc5,0x89,0x6f,0xb7,0x62,0x0e,0xaa,0x18,0xbe,0x1b],
    [0xfc,0x56,0x3e,0x4b,0xc6,0xd2,0x79,0x20,0x9a,0xdb,0xc0,0xfe,0x78,0xcd,0x5a,0xf4],
    [0x1f,0xdd,0xa8,0x33,0x88,0x07,0xc7,0x31,0xb1,0x12,0x10,0x59,0x27,0x80,0xec,0x5f],
    [0x60,0x51,0x7f,0xa9,0x19,0xb5,0x4a,0x0d,0x2d,0xe5,0x7a,0x9f,0x93,0xc9,0x9c,0xef],
    [0xa0,0xe0,0x3b,0x4d,0xae,0x2a,0xf5,0xb0,0xc8,0xeb,0xbb,0x3c,0x83,0x53,0x99,0x61],
    [0x17,0x2b,0x04,0x7e,0xba,0x77,0xd6,0x26,0xe1,0x69,0x14,0x63,0x55,0x21,0x0c,0x7d],
]
    
        SBOX = pd.DataFrame(data, index=['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f'],
                   columns =['0','1','2','3','4','5','6','7','8','9','a','b','c','d','e','f'])
        
        if state.shape == (4,4):
            transformed_state = [[SBOX.loc[hex(word)[-2],hex(word)[-1]]  
                         if len(hex(word))==4
                         else SBOX.loc['0',hex(word)[-1]]
                         for word in state[i] ] for i in range(4)]
        elif state.shape == (4,):
            transformed_state = [SBOX.loc[hex(word)[-2],hex(word)[-1]]  
                         if len(hex(word))==4
                         else SBOX.loc['0',hex(word)[-1]]
                         for word in state ]

        return np.array(transformed_state)
    
    
    def getRoundKeys(self, key):
        round_keys = np.array([[0x01,0x00,0x00,0x00],
                          [0x02,0x00,0x00,0x00],
                          [0x04,0x00,0x00,0x00],
                          [0x08,0x00,0x00,0x00],
                          [0x10,0x00,0x00,0x00],
                          [0x20,0x00,0x00,0x00],
                          [0x40,0x00,0x00,0x00],
                          [0x80,0x00,0x00,0x00],
                          [0x1b,0x00,0x00,0x00],
                          [0x36,0x00,0x00,0x00]])
        if len(key)!= 16:
            # enforce key 128 bits long
            raise Exception('Key must be a string of 16 characters')
        else:
            list_keys = [int(hex(ord(i)), 16) for i in key]
            key = np.transpose(np.array(list_keys).reshape(4,4)) # first 4 words of expanded key
        
            expanded_key = [key[:,i] for i in range(4)]  # we have word 0 to word 3
        
            for i in range(4, 44):
                temp  = expanded_key[i-1]
            
                fourth_offset = expanded_key[i-4]
            
            
            
            
                if i%4 == 0:
                    t = np.roll(temp, -1)
                    tp = self.subWord(t)
                    tmp = tp ^round_keys[int(i/4)-1] 
                
                
                    
                
                
                
                    expanded_key.append(tmp^fourth_offset)
                else:
                
                    x = temp^fourth_offset
                
                    expanded_key.append(temp^fourth_offset)

            expanded_key = np.reshape(expanded_key , (11,4,4))
            return expanded_key
    
    
    def encrypt(self):
        blocks = self.divideIntoBlocks(self._plaintext)
        roundKeys = self.getRoundKeys(self._key)
        
        encrypted = []
        for block in blocks: 
            
            
        
        
            block = block^roundKeys[0]
                
            for i in range(9):
                block = self.subWord(block)
                       # shift rows
                block[:,1] = np.roll(block[:,1], -1)
                block[:,2] = np.roll(block[:,2], -2)
                block[:,3] = np.roll(block[:,3], -3)
                       # mix columns
                block = self.mixColumns(block)
                       # add roundkey
                block = block^roundKeys[i+1]
        
            
            block = self.subWord(block)
        # shift rows
            block[:,1] = np.roll(block[:,1], -1)
            block[:,2] = np.roll(block[:,2], -2)
            block[:,3] = np.roll(block[:,3], -3)
        # add roundkey
            block = block^roundKeys[10]
            
            encrypted.append(block)
        return encrypted
        
    def decrypt(self, encrypted):
        '''
        the encrypted text may be in blocks of 128 bits or just a single block
        '''
        roundKeys = self.getRoundKeys(self._key)
        decrypted = []
        for encrypted_text in encrypted:
            
            encrypted_text = encrypted_text^roundKeys[10]
            for i in range(9, 0, -1):
                # inverse shift rows
                
                encrypted_text[:,1] = np.roll(encrypted_text[:,1], 1)
                encrypted_text[:,2] = np.roll(encrypted_text[:,2], 2)
                encrypted_text[:,3] = np.roll(encrypted_text[:,3], 3)
                    # inverse substitute words
                encrypted_text = self.invSubWord(encrypted_text)
                
                
                    # add round key
                encrypted_text = encrypted_text^roundKeys[i]
                
                
                    # inverse mix columns
                encrypted_text = self.invMixColumns(encrypted_text)
                
           
                # inverse shift rows
            encrypted_text[:,1] = np.roll(encrypted_text[:,1], 1)
            encrypted_text[:,2] = np.roll(encrypted_text[:,2], 2)
            encrypted_text[:,3] = np.roll(encrypted_text[:,3], 3)
                  
                # inverse sub words
            encrypted_text = self.invSubWord(encrypted_text)
            
                # add roundkey    
            encrypted_text = encrypted_text^roundKeys[0]
            
            decrypted.append(encrypted_text)
        
        return decrypted

In [722]:


# FIPS example versus new example


# plaintext is 00112233445566778899aabbccddeeff
plaintext = [chr(0x00), chr(0x11), chr(0x22), chr(0x33), chr(0x44), chr(0x55), chr(0x66), chr(0x77), chr(0x88), chr(0x99), chr(0xaa), chr(0xbb), chr(0xcc), chr(0xdd), chr(0xee), chr(0xff)]


txt = 'Dan Kioko erfgth'


# key is 000102030405060708090a0b0c0d0e0f
key = [chr(0x00), chr(0x01), chr(0x02), chr(0x03), chr(0x04), chr(0x05), chr(0x06), chr(0x07), chr(0x08), chr(0x09), chr(0x0a), chr(0x0b), chr(0x0c), chr(0x0d), chr(0x0e), chr(0x0f)]

In [723]:
AES = AES_CIPHER(plaintext=txt, key=key)


AES2 = AES_CIPHER(plaintext=plaintext, key=key)

In [724]:
encrypted = AES.encrypt()


encrypted2 = AES2.encrypt()

In [725]:

enc = np.array(encrypted).flatten()


[hex(i) for i in enc]

['0xb1',
 '0xd1',
 '0x6c',
 '0xd3',
 '0x15',
 '0xe4',
 '0xb',
 '0xc6',
 '0xac',
 '0x73',
 '0xd5',
 '0x3d',
 '0xe0',
 '0xb0',
 '0x40',
 '0x87']

In [727]:
enc2 = np.array(encrypted2).flatten()


[hex(i) for i in enc2]

['0x69',
 '0xc4',
 '0xe0',
 '0xd8',
 '0x6a',
 '0x7b',
 '0x4',
 '0x30',
 '0xd8',
 '0xcd',
 '0xb7',
 '0x80',
 '0x70',
 '0xb4',
 '0xc5',
 '0x5a']

In [734]:
decrypted = AES.decrypt(encrypted)
decrypted2 = AES.decrypt(encrypted2)
dec = np.array(decrypted).flatten()
dec2 = np.array(decrypted2).flatten()

In [735]:
''.join([chr(i) for i in dec])

'Dan Kioko erfgth'

In [736]:
[hex(i) for i in dec2]

['0x0',
 '0x11',
 '0x22',
 '0x33',
 '0x44',
 '0x55',
 '0x66',
 '0x77',
 '0x88',
 '0x99',
 '0xaa',
 '0xbb',
 '0xcc',
 '0xdd',
 '0xee',
 '0xff']